<a href="https://colab.research.google.com/github/mxa67690/neuralnetworks/blob/main/700756769_Neural_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Task 1: Save the trained model and use it to predict new text

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from tensorflow.keras.utils import to_categorical


In [ ]:
!pip install scikit-learn pandas matplotlib


In [ ]:
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.model_selection import train_test_split

import re

from sklearn.preprocessing import LabelEncoder

data = pd.read_csv('Data.csv')
# Keeping only the neccessary columns
data = data[['text','sentiment']]

data['text'] = data['text'].apply(lambda x: x.lower())
data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]', '', x)))

for idx, row in data.iterrows():
    row[0] = row[0].replace('rt', ' ')

max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['text'].values)
X = tokenizer.texts_to_sequences(data['text'].values)

X = pad_sequences(X)

embed_dim = 128
lstm_out = 196
def createmodel():
    model = Sequential()
    model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
    model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
    model.add(Dense(3,activation='softmax'))
    model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
    return model
# print(model.summary())

labelencoder = LabelEncoder()
integer_encoded = labelencoder.fit_transform(data['sentiment'])
y = to_categorical(integer_encoded)
X_train, X_test, Y_train, Y_test = train_test_split(X,y, test_size = 0.33, random_state = 42)

batch_size = 32
model = createmodel()
model.fit(X_train, Y_train, epochs = 1, batch_size=batch_size, verbose = 2)
score,acc = model.evaluate(X_test,Y_test,verbose=2,batch_size=batch_size)
print(score)
print(acc)
print(model.metrics_names)


<ipython-input-23-8314b59de35a>:17: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  row[0] = row[0].replace('rt', ' ')
<ipython-input-23-8314b59de35a>:17: FutureWarning: Series.__setitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To set a value by position, use `ser.iloc[pos] = value`
  row[0] = row[0].replace('rt', ' ')
/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


291/291 - 56s - 192ms/step - accuracy: 0.6447 - loss: 0.8310
144/144 - 5s - 35ms/step - accuracy: 0.6555 - loss: 0.7797
0.7797082662582397
0.6555264592170715
['loss', 'compile_metrics']


In [ ]:
# Adding  to the  existing code:

import numpy as np
import joblib

# --------- Saving the model and tokenizer ----------
model.save('sentiment_model.h5')
joblib.dump(tokenizer, 'tokenizer.pkl')
joblib.dump(labelencoder, 'labelencoder.pkl')

# --------- Predict on new input ----------
def clean_text(text):
    text = text.lower()
    text = re.sub('[^a-zA-z0-9\s]', '', text)
    text = text.replace('rt', ' ')
    return text

# Example input text given in the question
new_text = "A lot of good things are happening. We are respected again throughout the world, and that's a great thing. @realDonaldTrump"
new_text_cleaned = clean_text(new_text)
loaded_model = load_model('sentiment_model.h5')
tokenizer = joblib.load('tokenizer.pkl')
labelencoder = joblib.load('labelencoder.pkl')

# Tokenize and pad
seq = tokenizer.texts_to_sequences([new_text_cleaned])
padded = pad_sequences(seq, maxlen=X.shape[1])

# Predict
pred = loaded_model.predict(padded)
predicted_label = labelencoder.inverse_transform([np.argmax(pred)])
print("Predicted Sentiment:", predicted_label[0])


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 657ms/step
Predicted Sentiment: Negative


Task 2: Use GridSearchCV to find the best parameters

In [ ]:
!pip install scikit-learn scikeras keras tensorflow


In [ ]:
!pip install --upgrade tensorflow



In [ ]:
!pip install scikeras scikit-learn


In [ ]:

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder

import pandas as pd

import pandas as pd

from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
import re

# Load the data
data = pd.read_csv('Data.csv')
data = data[['text', 'sentiment']]

# Text preprocessing
data['text'] = data['text'].apply(lambda x: x.lower())
data['text'] = data['text'].apply(lambda x: re.sub('[^a-zA-Z0-9\s]', '', x))

# Tokenization
max_features = 2000
tokenizer = Tokenizer(num_words=max_features, split=' ')
tokenizer.fit_on_texts(data['text'].values)
X = tokenizer.texts_to_sequences(data['text'].values)
X = pad_sequences(X)

# Label Encoding
labelencoder = LabelEncoder()
integer_encoded = labelencoder.fit_transform(data['sentiment'])
y = to_categorical(integer_encoded)

# Split data into train and test sets
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.33, random_state=42)

# Function to create the model (for GridSearchCV)
def create_model(embed_dim=128, lstm_out=196):
    model = Sequential()
    model.add(Embedding(max_features, embed_dim, input_length=X.shape[1]))
    model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
    model.add(Dense(3, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# Wrap the model into KerasClassifier for use with GridSearchCV
model = KerasClassifier(build_fn=create_model, verbose=2)

# Define hyperparameters for GridSearchCV
param_grid = {
    'batch_size': [32, 64],
    'epochs': [1, 3],
    'embed_dim': [128, 256],
    'lstm_out': [196, 256]
}

# Setup GridSearchCV
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=3)

# Fit GridSearchCV
grid_result = grid.fit(X_train, Y_train)

# Print best results
print("Best Accuracy: ", grid_result.best_score_)
print("Best Parameters: ", grid_result.best_params_)

# Evaluate the model with the best found parameters
best_model = grid_result.best_estimator_
score, acc = best_model.score(X_test, Y_test, verbose=2, batch_size=32)
print(f"Test Accuracy: {acc}")


ModuleNotFoundError: No module named 'tensorflow.keras.wrappers.scikit_learn'

since the versions of python, scikilearn and tensor flow wrappers are not compatible to each other i am wrapping my own using sklearn.base

In [ ]:
from sklearn.base import BaseEstimator, ClassifierMixin

# Custom wrapper class
class KerasModelWrapper(BaseEstimator, ClassifierMixin):
    def __init__(self, embed_dim=128, lstm_out=196, batch_size=32, epochs=1):
        self.embed_dim = embed_dim
        self.lstm_out = lstm_out
        self.batch_size = batch_size
        self.epochs = epochs
        self.model = None

    def create_model(self):
        model = Sequential()
        model.add(Embedding(max_features, self.embed_dim, input_length=X.shape[1]))
        model.add(LSTM(self.lstm_out, dropout=0.2, recurrent_dropout=0.2))
        model.add(Dense(3, activation='softmax'))
        model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
        return model

    def fit(self, X, y):
        self.model = self.create_model()
        self.model.fit(X, y, batch_size=self.batch_size, epochs=self.epochs, verbose=2)
        return self

    def predict(self, X):
        return self.model.predict(X)

    def score(self, X, y):
        score, acc = self.model.evaluate(X, y, verbose=2, batch_size=self.batch_size)
        return acc

# Load the data
data = pd.read_csv('Data.csv')
data = data[['text', 'sentiment']]

# Text preprocessing
data['text'] = data['text'].apply(lambda x: x.lower())
data['text'] = data['text'].apply(lambda x: re.sub('[^a-zA-Z0-9\s]', '', x))

# Tokenization
max_features = 2000
tokenizer = Tokenizer(num_words=max_features, split=' ')
tokenizer.fit_on_texts(data['text'].values)
X = tokenizer.texts_to_sequences(data['text'].values)
X = pad_sequences(X)

# Label Encoding
labelencoder = LabelEncoder()
integer_encoded = labelencoder.fit_transform(data['sentiment'])
y = to_categorical(integer_encoded)

# Split data into train and test sets
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.33, random_state=42)

# Define the parameter grid
param_grid = {
    'batch_size': [32, 64],
    'epochs': [1, 3],
    'embed_dim': [128, 256],
    'lstm_out': [196, 256]
}

# Setup GridSearchCV with the custom wrapper
grid = GridSearchCV(estimator=KerasModelWrapper(), param_grid=param_grid, cv=3)

# Fit GridSearchCV
grid_result = grid.fit(X_train, Y_train)

# Print best results
print("Best Accuracy: ", grid_result.best_score_)
print("Best Parameters: ", grid_result.best_params_)

# Evaluate the model with the best found parameters
best_model = grid_result.best_estimator_
score = best_model.score(X_test, Y_test)
print(f"Test Accuracy: {score}")

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


194/194 - 38s - 198ms/step - accuracy: 0.6308 - loss: 0.8575
97/97 - 3s - 29ms/step - accuracy: 0.6624 - loss: 0.7667
194/194 - 48s - 249ms/step - accuracy: 0.6281 - loss: 0.8559
97/97 - 3s - 28ms/step - accuracy: 0.6556 - loss: 0.7922
194/194 - 36s - 188ms/step - accuracy: 0.6288 - loss: 0.8550
97/97 - 3s - 28ms/step - accuracy: 0.6719 - loss: 0.7780
194/194 - 52s - 266ms/step - accuracy: 0.6297 - loss: 0.8618
97/97 - 5s - 49ms/step - accuracy: 0.6530 - loss: 0.7786
194/194 - 50s - 257ms/step - accuracy: 0.6300 - loss: 0.8548
97/97 - 6s - 64ms/step - accuracy: 0.6501 - loss: 0.8012
194/194 - 50s - 257ms/step - accuracy: 0.6314 - loss: 0.8583
97/97 - 5s - 49ms/step - accuracy: 0.6700 - loss: 0.7845
Epoch 1/3
194/194 - 37s - 192ms/step - accuracy: 0.6328 - loss: 0.8568
Epoch 2/3
194/194 - 33s - 172ms/step - accuracy: 0.6980 - loss: 0.7075
Epoch 3/3
194/194 - 32s - 164ms/step - accuracy: 0.7419 - loss: 0.6222
97/97 - 3s - 30ms/step - accuracy: 0.6779 - loss: 0.7545
Epoch 1/3
194/194 - 44

Based on GridSearchCV, the best combination of parameters is:

batch_size=32

epochs=3

embed_dim=128

lstm_out=256

While the model performs reasonably well (with test accuracy around 70%), further hyperparameter tuning and model improvements can be made by testing with more sophisticated architectures or using different techniques for text preprocessing (e.g., including stop words removal, stemming, or lemmatization).